# Языковые модели на n-граммах

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* https://www.nltk.org/api/nltk.util.html
* https://web.stanford.edu/~jurafsky/slp3/3.pdf
* https://www.youtube.com/watch?v=QGT6XTeA3YQ

## Задачи для совместного разбора

1\. Выделите из текста n-граммы. 

In [29]:
text = """Вода это жидкость которая имеет свойство быть водой. 
Вода состоит из молекул, которые выглядят как вода"""

In [30]:
import nltk
# nltk.download('punkt_tab')

In [31]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
sents

['Вода это жидкость которая имеет свойство быть водой.',
 'Вода состоит из молекул, которые выглядят как вода']

In [32]:
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.util import ngrams

tokenizer = RegexpTokenizer(r"\w+")

unigrams = []
bigrams = []

for sent in sents:
    tokens = tokenizer.tokenize(sent.lower())
    print(tokens)
    unigrams.extend(ngrams(tokens, 1))
    bigrams.extend(ngrams(tokens, 2))

['вода', 'это', 'жидкость', 'которая', 'имеет', 'свойство', 'быть', 'водой']
['вода', 'состоит', 'из', 'молекул', 'которые', 'выглядят', 'как', 'вода']


In [33]:
unigrams

[('вода',),
 ('это',),
 ('жидкость',),
 ('которая',),
 ('имеет',),
 ('свойство',),
 ('быть',),
 ('водой',),
 ('вода',),
 ('состоит',),
 ('из',),
 ('молекул',),
 ('которые',),
 ('выглядят',),
 ('как',),
 ('вода',)]

In [34]:
bigrams

[('вода', 'это'),
 ('это', 'жидкость'),
 ('жидкость', 'которая'),
 ('которая', 'имеет'),
 ('имеет', 'свойство'),
 ('свойство', 'быть'),
 ('быть', 'водой'),
 ('вода', 'состоит'),
 ('состоит', 'из'),
 ('из', 'молекул'),
 ('молекул', 'которые'),
 ('которые', 'выглядят'),
 ('выглядят', 'как'),
 ('как', 'вода')]

Это же буквально TF из TF-IDF

2. Рассчитайте вероятности  $P(вода)$, $P(это|вода)$, $P(состоит|вода)$.

$$P(w_i) = \frac{C(w_i)}{N}$$
$$P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$$

In [35]:
x = ("вода", )
unigrams.count(x), len(unigrams)

(3, 16)

In [36]:
bigrams.count(("вода", "это"))/unigrams.count(("вода", ))

0.3333333333333333

In [37]:
bigrams.count(("вода", "состоит"))/unigrams.count(("вода", ))

0.3333333333333333

3. Рассчитайте вероятности  $P_L(вода)$, $P_L(это|вода)$, $P_L(состоит|вода)$.

$$P_L(w_i) = \frac{C(w_i)+1}{N+V}$$
$$P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$$

## Задачи для самостоятельного решения

In [38]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams
import random
from collections import Counter, defaultdict

<p class="task" id="1"></p>

1\. Считайте файл `data/moya-semia/Лучше кошки зверя нет 2.csv`. Получите список предложений из сообщений. Оберните каждое предложение в специальные токены `<s>` и `</s>`. Приведите предложения к нижнему регистру и удалите все символы, кроме букв, пробелов и введенных спец. токенов. После этого получите список слов (униграмм) и биграмм. Выведите 5 самых часто встречающихся униграмм и 5 самых часто встречающихся биграмм на экран.

- [x] Проверено на семинаре

In [39]:
messages = pd.read_csv("data/moya-semia/Лучше кошки зверя нет 2.csv", names=["source", "text", "extra"])["text"].dropna().tolist()


sents = []
for message in messages:
    sents.extend(sent_tokenize(message))


unigrams = []
bigrams = []

for sent in sents:
    tokens = ['<s>'] + re.sub(r"[^a-яёa-z\s]",'', sent.lower()).split() + ['</s>']
    unigrams.extend(ngrams(tokens, 1))
    bigrams.extend(ngrams(tokens, 2))
    

print("5 самых часто встречающихся униграмм")
unicounter = Counter(unigrams)
for i in unicounter.most_common(5):
    print(i)
    
print("5 самых часто встречающихся биграмм")
bicounter = Counter(bigrams)
for i in bicounter.most_common(5):
    print(i)

5 самых часто встречающихся униграмм
(('<s>',), 14447)
(('</s>',), 14447)
(('и',), 6355)
(('не',), 4377)
(('в',), 4110)
5 самых часто встречающихся биграмм
(('<s>', 'а'), 725)
(('<s>', 'и'), 718)
(('<s>', 'я'), 540)
(('<s>', 'но'), 463)
(('у', 'меня'), 332)


Понадобится потом

In [40]:
vocab = list(unicounter.keys()) 
vocab_size = len(vocab)
word_to_idx = {word: i for i, word in enumerate(vocab)}

context_dict = defaultdict(lambda : {})
for bigram, count in bicounter.items():
    prev_word = (bigram[0],) 
    next_word = (bigram[1],)
    context_dict[prev_word][next_word] = count

<p class="task" id="2"></p>

2\. Получите распределение вероятностей для униграм $P(w_i) = \frac{C(w_i)}{N}$, где $N$ - количество униграм, $C(w_i)$ - частота использования токена $w_i$. Получите распределение условных вероятностей для биграмм $P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$ ($C(w_{i-1} w_i)$ - частота использования словосочетания $w_{i-1}w_i$).

- [x] Проверено на семинаре

In [41]:
Pwi = dict()
N = sum(unicounter.values())
for k,v in unicounter.items():
    Pwi[k] = v/N

Pwi

{('<s>',): 0.07276729274644021,
 ('завтра',): 6.54789787294056e-05,
 ('мои',): 0.000533905518870538,
 ('котейки',): 5.036844517646585e-05,
 ('идут',): 7.555266776469877e-05,
 ('к',): 0.0038078544553408183,
 ('врачу',): 4.0294756141172676e-05,
 ('</s>',): 0.07276729274644021,
 ('а',): 0.00945415715962264,
 ('го',): 2.5184222588232925e-05,
 ('жду',): 9.066320131763853e-05,
 ('их',): 0.001626900779199847,
 ('домой',): 0.0010275162815999034,
 ('прошу',): 7.051582324705219e-05,
 ('помощи',): 7.051582324705219e-05,
 ('чем',): 0.0005641265859764175,
 ('кормить',): 0.00018636324715292363,
 ('лучше',): 0.0003374685826823212,
 ('корм',): 0.0005842739640470039,
 ('какой',): 0.00029213698202350193,
 ('наполнитель',): 0.0001611790245646907,
 ('использовать',): 2.5184222588232925e-05,
 ('коты',): 0.0006900476989175821,
 ('год',): 0.00034754227171761434,
 ('жили',): 0.00012592111294116462,
 ('на',): 0.019492588283292284,
 ('передержке',): 2.0147378070586338e-05,
 ('любым',): 3.022106710587951e-05,
 (

In [42]:
Pwiwi_1 = dict()

for k,v in bicounter.items():
    Pwiwi_1[k] = v/unicounter[(k[0],)]
    
Pwiwi_1

{('<s>', 'завтра'): 0.0006921852287672181,
 ('завтра', 'мои'): 0.15384615384615385,
 ('мои', 'котейки'): 0.05660377358490566,
 ('котейки', 'идут'): 0.2,
 ('идут', 'к'): 0.26666666666666666,
 ('к', 'врачу'): 0.009259259259259259,
 ('врачу', '</s>'): 0.375,
 ('<s>', 'а'): 0.050183429085623316,
 ('а', 'го'): 0.0010655301012253596,
 ('го', 'жду'): 0.4,
 ('жду', 'их'): 0.1111111111111111,
 ('их', 'домой'): 0.009287925696594427,
 ('домой', '</s>'): 0.25980392156862747,
 ('<s>', 'прошу'): 0.0004845296601370527,
 ('прошу', 'помощи'): 0.14285714285714285,
 ('помощи', '</s>'): 0.42857142857142855,
 ('<s>', 'чем'): 0.0004845296601370527,
 ('чем', 'кормить'): 0.017857142857142856,
 ('кормить', 'лучше'): 0.05405405405405406,
 ('лучше', 'корм'): 0.029850746268656716,
 ('корм', '</s>'): 0.12931034482758622,
 ('<s>', 'какой'): 0.0006229667058904963,
 ('какой', 'наполнитель'): 0.034482758620689655,
 ('наполнитель', 'использовать'): 0.0625,
 ('использовать', '</s>'): 0.4,
 ('<s>', 'коты'): 0.00159202602

<p class="task" id="3"></p>

3\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 токенов (слов), начинающийся с токена "мой". При генерации текста выбирайте слово с наибольшей вероятностью соответствующего биграмма. Сгенерируйте текст 3 раза и выведите полученные результаты на экран.

- [x] Проверено на семинаре

In [43]:
def predict(start_token="мой", length=20):
    current_word = (start_token,)
    generated_tokens = [start_token]
    
    for _ in range(length):

        real_next_words = context_dict.get(current_word, Counter())
        

        if not real_next_words:
            break
            
        candidates = list(real_next_words.keys())
        counts = list(real_next_words.values())
        
        next_token = candidates[np.argmax(counts)]
            
        
        generated_tokens.append(next_token[0])
        current_word = next_token
        
        if current_word == ('</s>',):
            break
            
    return " ".join(generated_tokens)

In [44]:
predict()

'мой кот </s>'

In [45]:
predict()

'мой кот </s>'

In [46]:
predict()

'мой кот </s>'

<p class="task" id="4"></p>

4\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 токенов (слов), начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Сгенерируйте текст 3 раза и выведите полученные результаты на экран.

- [x] Проверено на семинаре

In [47]:
def predict(start_token="мой", length=20):
    current_word = (start_token,)
    generated_tokens = [start_token]
    for i in range(length):
        real_next_words = context_dict.get(current_word, Counter())
        if not real_next_words:
            break
                
        candidates = list(real_next_words.keys())
        counts = list(real_next_words.values())

        next_token = random.choices(candidates, weights=counts, k=1)[0]

        
        generated_tokens.append(next_token[0])
        current_word = next_token
        
        if current_word == ('</s>',):
            break
        
    return  " ".join(generated_tokens)

In [48]:
predict()

'мой помер лежит её прихода то в нихещё момент лет смотрит </s>'

In [49]:
predict()

'мой трусишка </s>'

In [50]:
predict()

'мой сапог кому он дурак </s>'

<p class="task" id="5"></p>

5\. Получите распределение вероятностей для униграм, воспользовавшись сглаживанием Лапласа: $P_L(w_i) = \frac{C(w_i)+1}{N+V}$, где $V$ - количество уникальных униграмм. Получите распределение условных вероятностей для биграмм $P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$

- [x] Проверено на семинаре

In [51]:
PLwi = dict()
V = len(unicounter.keys())
for k,v in unicounter.items():
    PLwi[k] = (v+1)/(N+V)

PLwi

{('<s>',): 0.06282558594599295,
 ('завтра',): 6.087750576162108e-05,
 ('мои',): 0.00046527807974953257,
 ('котейки',): 4.783232595555942e-05,
 ('идут',): 6.957429229899552e-05,
 ('к',): 0.0032917337043962256,
 ('врачу',): 3.913553941818498e-05,
 ('</s>',): 0.06282558594599295,
 ('а',): 0.0081662825585946,
 ('го',): 2.609035961212332e-05,
 ('жду',): 8.261947210505719e-05,
 ('их',): 0.0014088794190546593,
 ('домой',): 0.0008914206200808801,
 ('прошу',): 6.52258990303083e-05,
 ('помощи',): 6.52258990303083e-05,
 ('чем',): 0.0004913684393616558,
 ('кормить',): 0.00016523894421011437,
 ('лучше',): 0.000295690742270731,
 ('корм',): 0.0005087620124364047,
 ('какой',): 0.00025655520285254596,
 ('наполнитель',): 0.00014349697786667827,
 ('использовать',): 2.609035961212332e-05,
 ('коты',): 0.0006000782710788363,
 ('год',): 0.0003043875288081054,
 ('жили',): 0.00011305822498586772,
 ('на',): 0.01683263034308823,
 ('передержке',): 2.17419663434361e-05,
 ('любым',): 3.043875288081054e-05,
 ('совет

In [52]:
PLwiwi_1 = dict()

for k,v in bicounter.items():
    PLwiwi_1[k] = (v+1)/(unicounter[(k[0],)]+V)
    
PLwiwi_1

{('<s>', 'завтра'): 0.00023975588491717523,
 ('завтра', 'мои'): 9.540164090822362e-05,
 ('мои', 'котейки'): 0.00022194743016582644,
 ('котейки', 'идут'): 9.541074324968991e-05,
 ('идут', 'к'): 0.00015899262274230477,
 ('к', 'врачу'): 0.00024853210724160425,
 ('врачу', '</s>'): 0.00012722241658980312,
 ('<s>', 'а'): 0.015823888404533564,
 ('а', 'го'): 9.006304413089163e-05,
 ('го', 'жду'): 9.542591767924168e-05,
 ('жду', 'их'): 9.538647419795873e-05,
 ('их', 'домой'): 0.00012596044841919638,
 ('домой', '</s>'): 0.0017068622182887126,
 ('<s>', 'прошу'): 0.0001743679163034002,
 ('прошу', 'помощи'): 9.539860718033517e-05,
 ('помощи', '</s>'): 0.00022259675008744873,
 ('<s>', 'чем'): 0.0001743679163034002,
 ('чем', 'кормить'): 9.51022349025202e-05,
 ('кормить', 'лучше'): 9.532888465204957e-05,
 ('лучше', 'корм'): 9.523809523809524e-05,
 ('корм', '</s>'): 0.0005071476116517164,
 ('<s>', 'какой'): 0.00021795989537925023,
 ('какой', 'наполнитель'): 9.52653138992093e-05,
 ('наполнитель', 'испол

<p class="task" id="6"></p>

6\.Воспользовавшись полученными после сглаживания вероятностями, сгенерируйте текст длиной не более 20 токенов (слов), начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Сгенерируйте текст 3 раза и выведите полученные результаты на экран.

- [x] Проверено на семинаре

In [53]:
def predict_laplace(start="мой", length=20):
    current_word = (start,)
    generated_text = [start]
    
    for _ in range(length):
        weights = np.ones(vocab_size)        
        real_continuations = context_dict.get(current_word, {})
        
        for next_word_tuple, count in real_continuations.items():
            if next_word_tuple in word_to_idx:
                idx = word_to_idx[next_word_tuple]
                weights[idx] += count 
        
        next_word_tuple = random.choices(vocab, weights)[0]
        
        generated_text.append(next_word_tuple[0])
        current_word = next_word_tuple
        
        if current_word == ('</s>',):
            break
            
    return ' '.join(generated_text)



for _ in range(3):
    print(predict_laplace())

мой маникюр держится разбитая подпрыгнуть дырка подстилки вняла удаленке агрессивной песенок заботиться природу причинык подкармливать наказания обувь шубемуж человеков ухнула надежда
мой сложнее московские вцепилась словои окей сытого стремительно нереальносколько похихикала пофигистичен такойа рыпнуться залезть милое сиганул посмотримвсех посидетьо впритык ага судьи
мой дороге дырки своея асей мужчинку поздравляю воздухжестокозато внезапные идёшь сисян многострадального громадным промолчу коту глубине повошкалась мурмуша уходить кошачьей выскальзывает


<p class="task" id="7"></p>

7\. Рассчитайте перплексию для текста "Котя пришел домой с хромой лапой" для четырех моделей: на 1/2-граммах и с/без использования сглаживания Лапласа. Сведите результат в таблицу. Повторите вычисления для текста "После пар я поеду кормить своего кота", используя доступные модели. Корректно обработайте ситуацию при работе с токенами, которых не было в обучающей выборке.

$Perplexity(W) = P(w_1w_2...w_N)^{-\frac{1}{N}}$

Для модели на униграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i)}$

Для модели на биграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i|w_{i-1})}$

- [ ] Проверено на семинаре

In [54]:
test1 = "После пар я поеду кормить своего кота".lower()
test2 = "Котя пришел домой с хромой лапой".lower()

In [55]:
def calculate_perplexity(text, model_type="unigram", smooth=False):
    cleaned_text = re.sub(r"[^a-яёa-z\s]", '', text)
    tokens = ['<s>'] + cleaned_text.split() + ['</s>']
    
    if model_type == "unigram":
        grams = list(ngrams(tokens, 1))
    else:
        grams = list(ngrams(tokens, 2))
    
    N_text = len(grams)
    if N_text == 0: return 0.0
    
    log_prob_sum = 0.0
    
    for gram in grams:
        prob = 0.0
        
        if model_type == "unigram":
            word = gram
            count = unicounter[word]
            
            if smooth:
                prob = (count + 1) / (N + V)
            else:
                prob = count / N if N > 0 else 0

        else:
            w_prev = (gram[0],)             
            count_bi = bicounter[gram]
            count_context = unicounter[w_prev]
            
            if smooth:
                prob = (count_bi + 1) / (count_context + V)
            else:
                if count_context > 0:
                    prob = count_bi / count_context
                else:
                    prob = 0.0

        if prob > 0:
            log_prob_sum += np.log(prob)
        else:
            return float('inf')

    return np.exp(-1/N_text * log_prob_sum)


In [56]:
texts = [
    test1,
    test2
]

results = []

for text in texts:
    row = {"Text": text}
    
    row["Uni"] = calculate_perplexity(text, "unigram", smooth=False)
    row["Uni (Laplace)"] = calculate_perplexity(text, "unigram", smooth=True)
    row["Bi"] = calculate_perplexity(text, "bigram", smooth=False)
    row["Bi (Laplace)"] = calculate_perplexity(text, "bigram", smooth=True)
    
    results.append(row)

df_res = pd.DataFrame(results)
df_res

,Text,Uni,Uni (Laplace),Bi,Bi (Laplace)
0,после пар я поеду кормить своего кота,1272.495052,1255.857113,inf,10664.562798
1,котя пришел домой с хромой лапой,506.447869,568.056724,27.532499,3319.210811
